In [16]:
from transformers import pipeline



In [2]:
generator = pipeline('text-generation', model = 'gpt2')
generator("I HATE MY JOB", max_length = 30, num_return_sequences=3)
## [{'generated_text': "Hello, I'm a language modeler. So while writing this, when I went out to meet my wife or come home she told me that my"},

/Users/ameezingwongstein/Library/Python/3.9/lib/python/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I HATE MY JOB"\n\n"Somewhat important" said the man, adding he was in fact a "wishlist"'},
 {'generated_text': 'I HATE MY JOB!"\n\nThe question about why these parents are so proud of their children: A. Is everything better than ever in'},
 {'generated_text': 'I HATE MY JOB AND HATE THAT MAN," said the father, who said he had recently completed a six-week course for the SAT'}]

## Idea 1
Train text generator on Amazon bad reviews


In [3]:
from datasets import load_dataset

dataset = load_dataset("amazon_us_reviews", "Digital_Video_Games_v1_00")

Dataset amazon_us_reviews downloaded and prepared to /Users/ameezingwongstein/.cache/huggingface/datasets/amazon_us_reviews/Digital_Video_Games_v1_00/0.1.0/17b2481be59723469538adeb8fd0a68b0ba363bbbdd71090e72c325ee6c7e563. Subsequent calls will reuse this data.


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 77.10it/s]


In [10]:
import pandas as pd

data_df = pd.DataFrame.from_dict(dataset['train'])
data_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,0,0,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,0,1,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,0,1,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,0,1,Five Stars,Awesome!,2015-08-31


In [11]:
bad_reviews = data_df.loc[data_df["star_rating"] == 1]
len(bad_reviews)

24970

In [14]:
bad_reviews["review_body"].head()

10    As has been written by so many others, I quick...
14                Lames purchase I almost never made...
24                                               Gross!
26           Not the best graphics :(<br />Dissapointed
30                                               Stolen
Name: review_body, dtype: object

# Cleaning Data

In [17]:
# a little sentiment analysis
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


KeyError: 0

In [26]:
bad_reviews.index[:10]

Int64Index([10, 14, 24, 26, 30, 35, 36, 43, 47, 62], dtype='int64')

In [31]:
cleaner_data = None

for index, bad_review in bad_reviews.iterrows():
    sentiment = sentiment_pipeline(bad_review["review_body"])[0]
    if (sentiment["label"] == "POSITIVE"):
        cleaner_data = bad_reviews.drop(index=index, axis = 0)
    elif sentiment["score"] < 0.8:
        cleaner_data = bad_reviews.drop(index=index, axis = 0)


RuntimeError: The size of tensor a (662) must match the size of tensor b (512) at non-singleton dimension 1

In [32]:
len(cleaner_data )

24969

# Retraining on top of transformers